# Numerical Approach

We use the minimization of a mean squared error function to calculate the linear regression model. 

The idea is to calculate the cost function (mse) and calculate the derivative of this function, the gradient, to understand how we can reduce this cost function. A pure numerical approach without any additional library to calculate any step. 


In [1]:
# Tradicional imports

import pandas as pd
import numpy as np

In [2]:
# Load the data
df = pd.read_csv('../data/linear_numeric_dummies.csv')

In [3]:
df.head()

,id,age,gender,target,S001_1,S001_2,S001_3,S001_4,S001_5,S002_1,...,R003_4,R003_5,T003_1,T003_2,T003_3,T003_4,T004_1,T004_2,T004_3,T004_4
0,1067163490,47,1,0,0,1,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
1,1073181139,60,1,2,0,0,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
2,1074271959,57,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,1074896711,74,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,1077476451,79,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0


In [4]:
X = np.array(df.iloc[:, 4:])
print(X)
print(X.shape)

[[0 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 1 0 ... 1 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
(203, 89)


In [5]:
y = np.array(df['target'])
print(y)
print(y.shape)

[0 2 0 0 0 0 0 3 0 0 0 0 0 4 0 0 0 0 0 4 0 1 4 0 1 0 0 1 0 0 0 0 0 0 0 0 0
 0 2 0 2 1 0 0 1 0 1 0 0 1 2 0 0 0 1 1 4 0 1 0 1 0 0 1 1 2 0 0 4 0 0 0 3 0
 0 0 0 5 1 2 0 0 0 0 0 1 1 0 0 3 0 5 0 1 0 5 2 1 3 0 5 0 3 1 1 2 1 0 0 5 0
 1 3 0 1 0 2 0 0 0 0 4 1 1 0 4 1 0 2 2 1 1 0 3 3 2 1 0 3 1 1 0 0 2 2 2 0 0
 0 0 5 0 1 3 2 0 0 4 2 5 2 0 2 0 1 1 1 0 0 0 0 1 0 1 5 0 1 1 0 0 1 5 0 5 2
 0 1 5 0 0 0 0 2 1 2 1 2 1 4 2 3 0 3]
(203,)


## Computing Cost
The term 'cost' in this assignment might be a little confusing since the data is housing cost. Here, cost is a measure how well our model is predicting the target price of the house. The term 'price' is used for housing data.

The equation for cost with one variable is:
  $$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2 \tag{1}$$ 
 
where 
  $$f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{2}$$
  
- $f_{w,b}(x^{(i)})$ is our prediction for example $i$ using parameters $w,b$.  
- $(f_{w,b}(x^{(i)}) -y^{(i)})^2$ is the squared difference between the target value and the prediction.   
- These differences are summed over all the $m$ examples and divided by `2m` to produce the cost, $J(w,b)$.  
>Note, in lecture summation ranges are typically from 1 to m, while code will be from 0 to m-1.


In [6]:
def compute_cost(X, y, w, b): 
    """
    compute cost
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    
    # number of training examples
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           #(n,)(n,) = scalar (see np.dot)
        cost = cost + (f_wb_i - y[i])**2       #scalar
    cost = cost / (2 * m)                      #scalar    
    return cost

<a name="toc_40291_2.1"></a>
## Gradient descent summary
So far in this course, you have developed a linear model that predicts $f_{w,b}(x^{(i)})$:
$$f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$
In linear regression, you utilize input training data to fit the parameters $w$,$b$ by minimizing a measure of the error between our predictions $f_{w,b}(x^{(i)})$ and the actual data $y^{(i)}$. The measure is called the $cost$, $J(w,b)$. In training you measure the cost over all of our training samples $x^{(i)},y^{(i)}$
$$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2\tag{2}$$ 


In lecture, *gradient descent* was described as:

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline
\;  w &= w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{3}  \; \newline 
 b &= b -  \alpha \frac{\partial J(w,b)}{\partial b}  \newline \rbrace
\end{align*}$$
where, parameters $w$, $b$ are updated simultaneously.  
The gradient is defined as:
$$
\begin{align}
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \tag{4}\\
  \frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \tag{5}\\
\end{align}
$$

Here *simultaniously* means that you calculate the partial derivatives for all the parameters before updating any of the parameters.

<a name="toc_40291_2.2"></a>
## Implement Gradient Descent
You will implement gradient descent algorithm for one feature. You will need three functions. 
- `compute_gradient` implementing equation (4) and (5) above
- `compute_cost` implementing equation (2) above (code from previous lab)
- `gradient_descent`, utilizing compute_gradient and compute_cost

Conventions:
- The naming of python variables containing partial derivatives follows this pattern,$\frac{\partial J(w,b)}{\partial b}$  will be `dj_db`.
- w.r.t is With Respect To, as in partial derivative of $J(wb)$ With Respect To $b$.


<a name="toc_40291_2.3"></a>
### compute_gradient
<a name='ex-01'></a>
`compute_gradient`  implements (4) and (5) above and returns $\frac{\partial J(w,b)}{\partial w}$,$\frac{\partial J(w,b)}{\partial b}$. The embedded comments describe the operations.

In [7]:
def compute_gradient(X, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """   
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):   
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

In [8]:
m, n = X.shape

# Compute and display gradient with w and b initialized to zeros
w_init = np.zeros(n)
b_init = 0.

#Compute and display gradient 
tmp_dj_db, tmp_dj_dw = compute_gradient(X, y, w_init, b_init)
print(f'dj_db at initial w,b: {tmp_dj_db}')
print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')

dj_db at initial w,b: -1.0788177339901477
dj_dw at initial w,b: 
 [-0.09852217 -0.24137931 -0.41871921 -0.27093596 -0.04926108 -0.08866995
 -0.20689655 -0.26600985 -0.43842365 -0.07881773 -0.10837438 -0.27093596
 -0.30049261 -0.30541872 -0.09359606 -0.03940887 -0.05418719 -0.22660099
 -0.38423645 -0.37438424 -0.06403941 -0.28571429 -0.25615764 -0.2364532
 -0.2364532  -0.37931034 -0.25615764 -0.16748768 -0.21674877 -0.0591133
 -0.33990148 -0.25123153 -0.26600985 -0.15763547 -0.06403941 -0.48768473
 -0.19211823 -0.26600985 -0.11330049 -0.01970443 -0.37438424 -0.2364532
 -0.23152709 -0.15763547 -0.07881773 -0.5320197  -0.54679803 -0.82758621
 -0.25123153 -0.66502463 -0.4137931  -0.03940887 -0.27093596 -0.29064039
 -0.38916256 -0.08866995 -0.08866995 -0.12315271 -0.29064039 -0.35960591
 -0.21674877 -0.04926108 -0.07881773 -0.24630542 -0.50246305 -0.20197044
 -0.31034483 -0.2364532  -0.33990148 -0.11330049 -0.07881773 -0.17241379
 -0.17241379 -0.2955665  -0.33004926 -0.10837438 -0.00492611 

<a name="toc_40291_2.5"></a>
###  Gradient Descent
Now that gradients can be computed,  gradient descent, described in equation (3) above can be implemented below in `gradient_descent`. The details of the implementation are described in the comments. Below, you will utilize this function to find optimal values of $w$ and $b$ on the training data.

In [9]:
def gradient_descent(X, 
                     y, 
                     w_in, 
                     b_in, 
                     cost_function, 
                     gradient_function, 
                     alpha, 
                     num_iters): 
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
    return w, b, J_history #return final w,b and J history for graphing

In [10]:
X_train = X
y_train = y

In [11]:
# initialize parameters
initial_w = np.zeros_like(w_init)
initial_b = 0.

# some gradient descent settings
iterations = 50000
alpha = 5.0e-5
# run gradient descent 
w_final, b_final, J_hist = gradient_descent(X_train, 
                                            y_train, 
                                            initial_w, 
                                            initial_b,
                                            compute_cost, 
                                            compute_gradient, 
                                            alpha, 
                                            iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")

Iteration    0: Cost     1.64   
Iteration 5000: Cost     1.00   
Iteration 10000: Cost     0.92   
Iteration 15000: Cost     0.86   
Iteration 20000: Cost     0.83   
Iteration 25000: Cost     0.80   
Iteration 30000: Cost     0.77   
Iteration 35000: Cost     0.75   
Iteration 40000: Cost     0.74   
Iteration 45000: Cost     0.72   
b,w found by gradient descent: 0.20,[ 0.05020234 -0.02147658  0.18959902 -0.01024417 -0.01188892  0.06719742
  0.02836987  0.0284807   0.0429553   0.02918839 -0.00937858 -0.03569719
  0.06155426  0.11289135  0.06682184  0.00497629 -0.00481676  0.15478523
  0.01194004  0.02930688  0.09608071  0.26085511  0.15035979 -0.03294992
 -0.278154    0.24705356  0.17227293  0.02977905 -0.06958557 -0.18332828
  0.06351828 -0.00772449  0.02375635  0.08541915  0.0312224   0.04863441
  0.00373397  0.12482515  0.01376016  0.005238   -0.01851178  0.03739909
  0.08581682  0.01297877  0.07850879  0.21798339 -0.0217917   0.27632215
 -0.08013047  0.20269736 -0.00650568 -0.02

In [12]:
m,_ = X_train.shape
for i in range(m):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

prediction: 1.22, target value: 0
prediction: 1.21, target value: 2
prediction: 0.89, target value: 0
prediction: 0.19, target value: 0
prediction: -0.13, target value: 0
prediction: 0.28, target value: 0
prediction: 1.29, target value: 0
prediction: 1.68, target value: 3
prediction: 1.05, target value: 0
prediction: 0.33, target value: 0
prediction: -0.18, target value: 0
prediction: 0.80, target value: 0
prediction: 0.50, target value: 0
prediction: 1.69, target value: 4
prediction: 0.48, target value: 0
prediction: 1.27, target value: 0
prediction: 0.73, target value: 0
prediction: 1.31, target value: 0
prediction: 1.21, target value: 0
prediction: 2.11, target value: 4
prediction: 0.06, target value: 0
prediction: 0.42, target value: 1
prediction: 2.03, target value: 4
prediction: 1.05, target value: 0
prediction: 0.94, target value: 1
prediction: 0.29, target value: 0
prediction: 0.02, target value: 0
prediction: 1.39, target value: 1
prediction: 0.13, target value: 0
prediction: 